In [1]:
import torch
import pandas as pd
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob
import os
from torch.distributions.multivariate_normal import *

In [2]:
# Noise
def noise(size, rho, gen_from_marked):
    if gen_from_marked:
        sigma_P = torch.tensor([[rho,0], [0,1/rho]])
        m = MultivariateNormal(torch.zeros(2), sigma_P)
        n = Variable(m.sample([size,50]))
        n = n.reshape(size,100)
        return n
    else:
        n = Variable(torch.randn(size, 100))
        #if torch.cuda.is_available(): 
        #   return n.cuda() 
        return n
    

In [3]:
def train_generator(optimizer, fake_data):
    # 2. Train Generator
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, real_data_target(prediction.size(0)))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, real_data_target(real_data.size(0)))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, fake_data_target(real_data.size(0)))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake

In [4]:
def real_data_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def fake_data_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [5]:
#parameters
lr = 0.0002
batch_size = 5
num_epochs = 5000
print_interval = 100
loss = nn.BCELoss()

In [ ]:
#Change noise to marked to run marked examples
gen_from = "noise"
fname = "population_obfuscation/" + gen_from
numCols = 2
base_dir = "original_data/population_obfuscation/target/theta-30/"
#counter to make sure everything ran
i = 0
if gen_from == "marked":
    gen_from_marked = True
else:
    gen_from_marked = False
    

#Iterates though all files under the base directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        #parses the file name for their info
        print(file)
        file_info = file.split('_')
        rho = file_info[1].split('-')[-1]
        theta = file_info[2].split('-')[-1]
        target_size = file_info[3].split('-')[-1]
        sample = file_info[4].split('-')[-1].split('.')[0]
        i = i + 1
        file_name = root +"/"+ file
        
        #initilizes the neural nets
        discriminator = DiscriminatorNet(numCols)
        generator = GeneratorNet(numCols, float(rho), gen_from_marked)
        if torch.cuda.is_available():
            discriminator.cuda()
            generator.cuda()
        d_optimizer = optim.Adam(discriminator.parameters(), lr=lr)
        g_optimizer = optim.Adam(generator.parameters(), lr=lr)
        
        #inililize the generator and the discriminator
        d_error_plt = [0]
        g_error_plt = [0]
        generated_points = []
        
        #load in the data
        database = DataSet(csv_file=file_name, root_dir=".")
        data_loader = torch.utils.data.DataLoader(database, batch_size=batch_size, shuffle=True)
        num_batches = len(data_loader)
        
        #Display progress
        #if i%100 == 0:
         #   print("On trial #" + i + " of 4800")
    
        #train the GAN
        for epoch in range(num_epochs):
            if((epoch % 100) == 0):
                print("Epoch ", epoch)

            for n_batch, real_batch in enumerate(data_loader):
                # 1. Train Discriminator
                real_data = Variable(real_batch).float()
                if torch.cuda.is_available(): 
                    real_data = real_data.cuda()
                # Generate fake data
                fake_data = generator(noise(real_data.size(0),float(rho), gen_from_marked)).detach()
                # Train D
                d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer,
                                                                real_data, fake_data)

                # 2. Train Generator
                # Generate fake data
                fake_data = generator(noise(real_batch.size(0),float(rho), gen_from_marked))
                generated_points.append(fake_data)
                # Train G
                g_error = train_generator(g_optimizer, fake_data)

                # Display Progress

                #if (n_batch) % print_interval == 0:
            filename = "results/" + fname + "/" + str(epoch) + "_" + str(n_batch) + ".txt"
            #print(filename)
            file = open(filename, "w")
            #print("Discriminator error: ", d_error)
            #print("Generator error: ", g_error)
            #print("")

            file.write("Discriminator error: " + str(d_error) + "\n")
            file.write("Generator error: " + str(g_error) + "\n")
            file.write("Points: " + str(fake_data) + "\n\n\n")

            d_error_plt.append(d_error)
            g_error_plt.append(g_error)
            
        #save the models
        output_dir = "theta-" + theta + "/rho-" + rho + "/size-" + target_size + "/"
        output_name = gen_from + "_rho-" + rho + "_theta-" + theta + "_size-" + target_size + "_sample-" + sample
        torch.save({
            'epoch': epoch,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': g_optimizer.state_dict(),
            'loss': loss
            }, "models/population_obfuscation/" + gen_from + "/generator/" + output_dir + output_name + ".pt")

        torch.save({
            'epoch': epoch,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': d_optimizer.state_dict(),
            'loss': loss
            }, "models/population_obfuscation/" + gen_from + "/discriminator/" + output_dir + output_name + ".pt")
        
        #Save error info
        #plt.plot(d_error_plt, 'b')
        #plt.plot(g_error_plt, 'r')
        filename = "results/" + fname + "/" + "error_growth.txt"
        file = open(filename, "w")
        file.write("Discriminator error: " + str(d_error_plt) + "\n")
        file.write("\n\n\n")
        file.write("Generator error: " + str(g_error_plt) + "\n")
        file.close()

        #plt.savefig('images/'+ fname + "/"+ 'error.png')
        #plt.show()
        #plt.clf()
        
        #Save a CSV for data analysis
        try:
            checkpoint= torch.load("models/population_obfuscation/" + gen_from + "/generator/" + output_dir + output_name + ".pt", map_location='cuda')
        except:
            checkpoint= torch.load("models/population_obfuscation/" + gen_from + "/generator/" + output_dir + output_name + ".pt", map_location='cpu')
        generator = GeneratorNet(numCols, float(rho), gen_from_marked)
        generator.load_state_dict(checkpoint['model_state_dict'])

        size = 500
        new_data = generator.create_data(size)
        df = pd.DataFrame(new_data, columns=['V1', 'V2'])
        #Changes the name to be easier to read
        df.to_csv("fake_data/population_obfuscation/" + gen_from + "/" + output_dir + output_name + ".csv", index=False)


target_rho-1.5_theta-30_size-10_sample-1.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-10.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  27

Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-20.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-21.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2

Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-32.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-33.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1

Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-44.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-45.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1

Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-56.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-1.5_theta-30_size-10_sample-57.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1

...

Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-10_sample-82.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-10_sample-83.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch 

Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-10_sample-94.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-10_sample-95.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Ep

Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-100_sample-15.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-100_sample-1

Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-100_sample-27.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch

Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch  4300
Epoch  4400
Epoch  4500
Epoch  4600
Epoch  4700
Epoch  4800
Epoch  4900
target_rho-2.5_theta-30_size-100_sample-39.csv
Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
Epoch  1000
Epoch  1100
Epoch  1200
Epoch  1300
Epoch  1400
Epoch  1500
Epoch  1600
Epoch  1700
Epoch  1800
Epoch  1900
Epoch  2000
Epoch  2100
Epoch  2200
Epoch  2300
Epoch  2400
Epoch  2500
Epoch  2600
Epoch  2700
Epoch  2800
Epoch  2900
Epoch  3000
Epoch  3100
Epoch  3200
Epoch  3300
Epoch  3400
Epoch  3500
Epoch  3600
Epoch  3700
Epoch  3800
Epoch  3900
Epoch  4000
Epoch  4100
Epoch  4200
Epoch